In [1]:
import cv2
import numpy as np
import pandas as pd

In [2]:
def get_full_parking_lots(points,car_locations):
    points = points.iloc[2:,:].sort_values(by='x1').reset_index().iloc[:,1:]
    points = points[points.index % 2 == 0]
    x_coordinates = points.x1.values
    full_parking_lots = []
    
    for i in range(len(x_coordinates)):
        for car in car_locations:
            if x_coordinates[i] <  car[0] and x_coordinates[i+1] > car[0]:
                full_parking_lots.append(i+1)
                
    return full_parking_lots

In [3]:
def get_area(w,h):
    return w*h if w*h >500 else -1

In [5]:
ALL_PARKING_LOTS = [1,2,3,4,5]

car_classifier = cv2.CascadeClassifier('haarcascade/haarcascade_car.xml')

image = cv2.imread('data/parking_lot_new.jpg')
cv2.imshow("Original", image)
cv2.waitKey(0)

# Grayscale and Canny Edges extracted
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 70, 200, apertureSize = 3)

cars = car_classifier.detectMultiScale(image, 1.11,0)

cv2.imshow("edges", edges)
cv2.waitKey(0)

# Run HoughLines using a rho accuracy of 1 pixel
# theta accuracy of np.pi / 180 which is 1 degree
# Our line threshold is set to 240 (number of points on line)
lines = cv2.HoughLines(edges, 1, np.pi/90, 200)

points = np.array([])
car_locations = np.array([])

# We iterate through each line and convert it to the format
# required by cv.lines (i.e. requiring end points)
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    points = np.append(points,[x1,x2,y1,y2]).reshape(-1,4)

points_df =pd.DataFrame(points, columns = ['x1','x2','y1','y2'])
points_df.sort_values(by='x1', inplace=True)

for (x,y,w,h) in cars:
    if(get_area(w,h)) != -1:
#         cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
        cv2.putText(image, org = (x+40,y+50), text = "Full", fontFace = cv2.FONT_HERSHEY_SIMPLEX,
               fontScale = 1.0, color = (0,255,255), thickness = 2)
        car_locations = np.append(car_locations,[(x+x+w)/2, (y+y+h)/2]).reshape(-1,2)


for p in range(0,len(points_df),2):
    x1,x2,y1,y2 = points_df.iloc[p,:].values
    cv2.line(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    if p != 0 :
        cv2.putText(image, org = (int((x1+x2) / 2 + 65), int(image.shape[0] / 2)), text = str(int(p/2)), fontFace = cv2.FONT_HERSHEY_SIMPLEX,
               fontScale = 1.0, color = (0,0,255), thickness = 2)
        
print("FULL PARKING LOTS:")
full_parking_lots = get_full_parking_lots(points_df,car_locations)
print(set(full_parking_lots))
print()
print("EMPTY PARKING LOTS:")
print(set(ALL_PARKING_LOTS) - set(full_parking_lots))
        
cv2.imshow('Final', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

FULL PARKING LOTS:
{1, 2}

EMPTY PARKING LOTS:
{3, 4, 5}
